In [22]:
import os
import tensorflow as tf
import random
import shutil
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam, RMSprop
print(tf.__version__)

2.1.0


In [7]:
BOY_SAMPLE_DIR = 'sample/boy'
GIRL_SAMPLE_DIR = 'sample/girl'

boys = os.listdir(BOY_SAMPLE_DIR)
girls = os.listdir(GIRL_SAMPLE_DIR)

In [8]:
training_size = 90

boys_train = random.sample(boys, len(boys)*training_size//100)
girls_train = random.sample(girls, len(girls)*training_size//100)

boys_test = [item for item in boys if item not in boys_train]
girls_test = [item for item in girls if item not in girls_train]

In [9]:
BASE_DIR = 'image'
TRAIN_DIR = os.path.join(BASE_DIR, 'train')
VAL_DIR = os.path.join(BASE_DIR, 'val')

BOY_TRAIN_DIR = os.path.join(TRAIN_DIR, 'boy')
GIRL_TRAIN_DIR = os.path.join(TRAIN_DIR, 'girl')
BOY_VAL_DIR = os.path.join(VAL_DIR, 'boy')
GIRL_VAL_DIR = os.path.join(VAL_DIR, 'girl')

In [17]:
for image in tqdm(boys_train):
    src = os.path.join(BOY_SAMPLE_DIR, image)
    des = BOY_TRAIN_DIR
    shutil.copy(src, des)

for image in tqdm(girls_train):
    src = os.path.join(GIRL_SAMPLE_DIR, image)
    des = GIRL_TRAIN_DIR
    shutil.copy(src, des)

for image in tqdm(boys_test):
    src = os.path.join(BOY_SAMPLE_DIR, image)
    des = BOY_VAL_DIR
    shutil.copy(src, des)

for image in tqdm(girls_test):
    src = os.path.join(GIRL_SAMPLE_DIR, image)
    des = GIRL_VAL_DIR
    shutil.copy(src, des)

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:02<00:00, 90.63it/s]


In [21]:
train_datagen = ImageDataGenerator(rescale = 1.255)
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    batch_size = 32,
    target_size= (54, 54),
    class_mode = 'binary'
)

val_datagen =  ImageDataGenerator(rescale = 1.255)
val_generator = val_datagen.flow_from_directory(
    VAL_DIR,
    batch_size = 32,
    target_size= (54, 54),
    class_mode = 'binary'
)

Found 3600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [30]:
model = tf.keras.Sequential()
model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(54, 54, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 52, 52, 16)        448       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 26, 26, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 32)        4640      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                294976    
_________________________________________________________________
dropout (Dropout)            (None, 64)               

In [31]:
model.compile(optimizer=RMSprop(lr=0.01), loss='binary_crossentropy', metrics=['accuracy'])

In [32]:
history = model.fit(
    train_generator,
    epochs = 10,
    validation_data = val_generator,
    verbose = 1
)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 113 steps, validate for 13 steps
Epoch 1/10
113/113 [==============================] - 11s 102ms/step - loss: 299.9834 - accuracy: 0.4981 - val_loss: 0.6989 - val_accuracy: 0.5000
Epoch 2/10
113/113 [==============================] - 10s 88ms/step - loss: 0.8991 - accuracy: 0.5053 - val_loss: 0.6951 - val_accuracy: 0.5075
Epoch 3/10
113/113 [==============================] - 10s 92ms/step - loss: 0.8104 - accuracy: 0.5108 - val_loss: 0.7045 - val_accuracy: 0.5025
Epoch 4/10
113/113 [==============================] - 11s 93ms/step - loss: 2.2258 - accuracy: 0.5197 - val_loss: 0.7136 - val_accuracy: 0.4975
Epoch 5/10
113/113 [==============================] - 10s 89ms/step - loss: 3.6818 - accuracy: 0.5156 - val_loss: 0.6749 - val_accuracy: 0.5225
Epoch 6/10
113/113 [==============================] - 10s 87ms/step - loss: 2.5129 - accuracy: 0.5169 - val_loss: 0.7087 - val_accuracy: 0.5050
Epoch 7/10
113/113 [====================